# Sulphur Isotope Analysis
This notebook takes the raw $\mathrm{\delta^{34}S}$ (`d34S`) data from the mass spectrometer, and corrects the drift in the isotopic composition of the standards using linear interpolation. Then, the true $\mathrm{\delta^{34}S}$ of the samples are calculated relative to the standards.

## Version 2
This uses the second run of sulphur isotope data, collected from the same samples in March 2021, when the mass spec was behaving a bit better.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import scipy.interpolate as itp
import ipywidgets as wdg
from plotly_default import go, graph_config, sel_trace# Custom graph style

In [2]:
# True d34S values of standards
true_std = {'NBS': 20.3,
            'S3' : -32.49}

standards = list(true_std.keys()) # list of standard names

In [3]:
# Read in MS data
isotope_df = pd.read_csv('./Data/S_isotopes_raw_v2.csv')
isotope_df.head(20)

,run_no,name,data_type,group,d34S_raw
0,1,NBS,standard,0,21.181
1,2,NBS,standard,0,21.236
2,3,NBS,standard,0,21.475
3,4,NBS,standard,0,21.258
4,5,NBS,standard,0,21.423
5,6,NBS,standard,0,21.576
6,8,S3,standard,0,-26.387
7,9,S3,standard,0,-27.226
8,10,S3,standard,0,-27.837
9,12,KC-2-0,normal,1,-15.482


In [4]:
# Set up dictionary for average of each cluster of standards
cal_points = {std: {'avg_run':[], 'd34S_raw': [], 'd34S_corrected': []} for std in standards}

for standard in standards:
    # selects all groups of standard
    for sample_group in range(isotope_df.group.min(), isotope_df.group.max()+1, 2):
        group_df = isotope_df[(isotope_df.group == sample_group) & (isotope_df.name == standard)]
        # caluclate average run number
        cal_points[standard]['avg_run'].append(group_df.run_no.mean())
        # calculate average d34S
        cal_points[standard]['d34S_raw'].append(group_df.d34S_raw.mean())
        # If this all works, these d34S values should correct to the true values
        cal_points[standard]['d34S_corrected'].append(true_std[standard])

In [5]:
# Interpolate standard d34S
for standard in standards:
    # Interpolate linearly between calibration points
    itp_fn = itp.interp1d(x=cal_points[standard]['avg_run'],
                          y=cal_points[standard]['d34S_raw'],
                          fill_value='extrapolate')
    
    # Evaluate interpolation function at each data point
    itp_values = itp_fn(isotope_df.run_no)
    isotope_df[standard+'_itp'] = itp_values

![S isotope correction](images/Isotope_correction.png)

In [6]:
Delta_true = true_std['NBS'] - true_std['S3']

d34Ss_corrected = [] # empty list to store corrected d34S

# Correct isotope values
for sample in isotope_df.iterrows():
    Delta_NBS = sample[1].NBS_itp - sample[1].d34S_raw
    Delta_S3 = sample[1].d34S_raw - sample[1].S3_itp
    Delta_S3_corrected = Delta_S3 * Delta_true / (Delta_NBS + Delta_S3)
    d34Ss_corrected.append(true_std['S3'] + Delta_S3_corrected)
    
isotope_df['d34S_corrected'] = d34Ss_corrected

In [7]:
# Calculate errors
std_NBS = isotope_df[isotope_df.name == 'NBS'].d34S_raw.std() # σ of NBS samples. Normalised n-1 by default.
std_S3 = isotope_df[isotope_df.name == 'S3'].d34S_raw.std()

av_err = (std_NBS + std_S3)/2

isotope_df['d34S_err'] = av_err

In [8]:
isotope_df.head(20)

,run_no,name,data_type,group,d34S_raw,NBS_itp,S3_itp,d34S_corrected,d34S_err
0,1,NBS,standard,0,21.181,21.360171,-27.262079,20.105470,0.538145
1,2,NBS,standard,0,21.236,21.359370,-27.248069,20.166015,0.538145
2,3,NBS,standard,0,21.475,21.358568,-27.234059,20.426490,0.538145
3,4,NBS,standard,0,21.258,21.357766,-27.220050,20.191584,0.538145
4,5,NBS,standard,0,21.423,21.356964,-27.206040,20.371784,0.538145
5,6,NBS,standard,0,21.576,21.356162,-27.192030,20.539046,0.538145
6,8,S3,standard,0,-26.387,21.354558,-27.164010,-31.644584,0.538145
7,9,S3,standard,0,-27.226,21.353756,-27.150000,-32.572716,0.538145
8,10,S3,standard,0,-27.837,21.352954,-27.135990,-33.253191,0.538145
9,12,KC-2-0,normal,1,-15.482,21.351350,-27.107970,-19.825048,0.538145


In [9]:
errorbar = dict(type='constant', value=av_err, thickness=1.5)


std_plot = go.FigureWidget();

for standard, colour in zip(standards, ['RoyalBlue', 'green']):
    std_df = isotope_df[isotope_df.name==standard]
    std_plot.add_trace(go.Scatter(error_y = errorbar,
                                  name=standard, marker_color=colour))

    std_plot.add_trace(go.Scatter(name=standard+' (average)',
                                  marker_color=colour, marker_symbol='square',
                                  opacity=0.5))

    std_plot.add_hline(true_std[standard], line_width=2, line_color=colour, line_dash='dash')

std_plot.update_layout(xaxis_title = 'Run number',
                       xaxis_rangemode = 'tozero',
                       yaxis_title = 'δ<sup>34</sup>S (‰)',
                       width=900, height=500)   

std_plot.add_trace(go.Scatter(error_y = errorbar,
                              name='Samples', marker_color='GoldenRod',
                              mode='markers+lines'))


# std_plot.show(config=graph_config)

FigureWidget({
    'data': [{'error_y': {'thickness': 1.5, 'type': 'constant', 'value': 0.5381447352028924},
 …

In [10]:
@wdg.interact(corrected = False)
def update_std_plot(corrected):
    if corrected:
        plot_col = 'd34S_corrected'
    else:
        plot_col = 'd34S_raw'
    
    # Set standards x and y
    for standard, colour in zip(standards, ['RoyalBlue', 'green']):
        std_df = isotope_df[isotope_df.name==standard]
        std_points = sel_trace(std_plot, standard)
        std_points.x = std_df.run_no
        std_points.y = std_df[plot_col]
        
        # Set standard averages x and y
        avg_points = sel_trace(std_plot, standard+' (average)')
        x=cal_points[standard]['avg_run']
        y=cal_points[standard][plot_col]        
    
    
    # Set samples x and y
    sample_isotopes=isotope_df[isotope_df.data_type.isin(['normal', 'duplicate', 'repeat'])]
    samples_points = sel_trace(std_plot, 'Samples')
    samples_points.x = sample_isotopes['run_no']
    samples_points.y = sample_isotopes[plot_col]
    
std_plot

interactive(children=(Checkbox(value=False, description='corrected'), Output()), _dom_classes=('widget-interac…

FigureWidget({
    'data': [{'error_y': {'thickness': 1.5, 'type': 'constant', 'value': 0.5381447352028924},
 …

In [11]:
# Remove standards, remove unnecessary columns and average repeats of same sample
S_corrected = isotope_df[isotope_df.data_type != 'standard'].loc[:,['name', 'd34S_corrected', 'd34S_err']].rename(columns={'d34S_corrected': 'd34S'}).groupby(['name']).mean()
S_corrected

,d34S,d34S_err
name,,
KC-2-0,-19.825048,0.538145
KC-2-1,-21.990780,0.538145
KC-2-10,-9.997303,0.538145
KC-2-11,-18.898315,0.538145
KC-2-12,-5.549121,0.538145
KC-2-13,-16.232995,0.538145
KC-2-14,-18.611586,0.538145
KC-2-14.5,-14.053474,0.538145
KC-2-15,-20.798288,0.538145


In [12]:
S_corrected.to_csv('Data/S_isotopes_processed_v2.csv')